In [1]:
import theano
import theano.tensor as th
import numpy as np
import math

In [2]:
import sys
sys.path.append('..')

In [3]:
import rnn.theano.model
import rnn.theano.lstm
import rnn.theano.crossent
import rnn.theano.solvers
import rnn.theano.softmax
from rnn.initializers import orthogonal
reload(rnn.theano.lstm)
reload(rnn.theano.model)
reload(rnn.theano.crossent)
reload(rnn.theano.solvers)
reload(rnn.theano.softmax)

<module 'rnn.theano.softmax' from '../rnn/theano/softmax.pyc'>

### Construct the model

In [4]:
n = 4

In [5]:
x = th.imatrix()
y = th.imatrix()

k,b = x.shape

lstm_units = 100
#s0 = th.matrix()
lstm = rnn.theano.lstm.LSTM(n, lstm_units)
y0 = th.zeros((b, lstm_units))
c0 = th.zeros((b, lstm_units))
lstm_out, _ = lstm.scanl(y0, c0, x)

wlin = np.random.randn(lstm_units+1, n).astype(theano.config.floatX)
wlin[0] = 0
orthogonal(wlin[1:])
wlin = theano.shared(wlin)
lin_out = th.dot(lstm_out.reshape((k*b,lstm_units)), wlin[1:]) + wlin[0]
lin_out = lin_out.reshape((k,b,n))

yh = rnn.theano.softmax.softmax(lin_out)
#print theano.printing.debugprint(yh)
err = th.sum(rnn.theano.crossent.crossent(yh, y))
acc = th.sum(th.eq(th.argmax(yh, axis=2), y))
count = y.size

solver = rnn.theano.solvers.RMSprop(0.01, decay=rnn.theano.solvers.GeomDecay(0.995))
model = rnn.theano.model.Model([lstm.weights, wlin], yh, x, y
                               , err, acc, count, solver=solver)

### Generate data

In [6]:
class Minibatch(object):
    def __init__(self, data, size=256):
        self.data = data
        self.size = size
        
    def __len__(self):
        return int(math.ceil(len(self.data)/float(self.size)))
    
    def __iter__(self):
        n = len(self.data)
        p = np.random.permutation(n)
        for i in xrange(0, n, self.size):
            yield self.data[i:i+self.size]
            
class Loader(object):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        if np.issubdtype(self.data.dtype, int):
            return self.data.shape[-1]
        return self.data.shape[-2]
    
    def __getitem__(self, i):
        return self.data[:-1,i], self.data[1:,i]

In [7]:
length, samples = 200, 100
batches = 20
D = np.random.randint(0, n, (length,samples)).astype(np.int32)
D = Minibatch(Loader(D), size=batches)

In [8]:
len(D)

5

In [9]:
from rnn.progress import progress_bar

max_iters = 500
err_ = 0
acc_ = 0
n_ = 0
bar = progress_bar()
next(bar)
for iters, (err,acc,n) in model.fit(D):
    h = '{}/{}:'.format(int(iters), max_iters)
    p = iters % 1
    err_ += err
    acc_ += acc
    n_ += n
    bar.send((h, p, {'error':err_/n_, 'accurracy':float(acc_)/n_}))
    if iters >= max_iters:
        break
    if p == 0:
        err_, acc_, n_ = 0, 0, 0

0/500: accurracy=0.241959798995, error=1.38738941333 
    [########                                ]  20.00%, eta 00:00:51
1/500: accurracy=0.246633165829, error=1.39067024446 
2/500: accurracy=0.247135678392, error=1.3881992384 
3/500: accurracy=0.250854271357, error=1.38817646735 
4/500: accurracy=0.256633165829, error=1.38625875004 
5/500: accurracy=0.257989949749, error=1.38627093905 
6/500: accurracy=0.260150753769, error=1.38569260436 
7/500: accurracy=0.261557788945, error=1.38579801091 
8/500: accurracy=0.259748743719, error=1.38552693395 
9/500: accurracy=0.26216080402, error=1.38553130353 
10/500: accurracy=0.260904522613, error=1.3853778538 
11/500: accurracy=0.266783919598, error=1.38524255638 
12/500: accurracy=0.266030150754, error=1.38491690142 
13/500: accurracy=0.263969849246, error=1.38467380411 
14/500: accurracy=0.263969849246, error=1.38465702191 
15/500: accurracy=0.259396984925, error=1.3861265225 
16/500: accurracy=0.26351758794, error=1.38506250273 
17/500: acc